## colab settings

In [1]:
import os

prj_name = 'movielens'
prj_path = '/content/drive/My Drive/colab/study/recommender_system/'\
        + prj_name + '/'
os.chdir(prj_path + 'notebooks/')

## settings

In [2]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append('..')

# main

In [3]:
rawpath = '../data/raw/'
interimpath = '../data/interim/'

## save raw data

In [4]:
from urllib.request import urlretrieve

url = 'http://files.grouplens.org/datasets/movielens/ml-20m.zip'
fn = 'ml-20m.zip'
urlretrieve(url, rawpath + fn)

('../data/raw/ml-20m.zip', <http.client.HTTPMessage at 0x7fd9eea52ef0>)

In [5]:
import zipfile

with zipfile.ZipFile(rawpath + fn) as f:
    f.extractall(rawpath)

os.remove(rawpath + fn)

## save raw dataset

In [6]:
import torch

In [7]:
import pandas as pd

ratings = pd.read_csv(rawpath + 'ml-20m/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [8]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
ratings['movieId'] = enc.fit_transform(ratings['movieId'])
ratings['userId'] = enc.fit_transform(ratings['userId'])

In [9]:
n_um = (len(ratings['userId'].unique()), len(ratings['movieId'].unique()))
torch.save(n_um, interimpath + 'n_um.pt')

In [10]:
X = ratings[['userId', 'movieId']].values
y = ratings[['rating']].values

In [11]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.1)
train_X, val_X, train_y, val_y =\
        train_test_split(train_X, train_y, test_size=0.2)

In [12]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(torch.tensor(train_X, dtype=torch.int64),
        torch.tensor(train_y, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(val_X, dtype=torch.int64),
        torch.tensor(val_y, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(test_X, dtype=torch.int64),
        torch.tensor(test_y, dtype=torch.float32))

In [13]:
torch.save(train_dataset, interimpath + 'train_dataset.pt')
torch.save(val_dataset, interimpath + 'val_dataset.pt')
torch.save(test_dataset, interimpath + 'test_dataset.pt')